In [84]:
import numpy as np
import random

In [85]:
#Brute-force method for validation
def brute_force_knapsack(A, b, c):
    n = len(c)
    best_value = 0
    best_solution = None

    #Iterate over all possible combinations of x
    for x in range(2 ** n):
        solution = [int(i) for i in bin(x)[2:].zfill(n)]

        #Check constraints
        if np.all(np.dot(A, solution) <= b):
            value = np.dot(c, solution)
            if value > best_value:
                best_value = value
                best_solution = solution

    return best_value, best_solution

In [86]:
def generate_problem_instance(m, n, seed=42):
    np.random.seed(seed)
    A = np.random.randint(1, 50, size=(m,n))
    b = np.random.randint(10, 100, size=m)
    c = np.random.randint(1, 100, size=n)
    return A, b, c

In [87]:
m, n = 10, 15
A, b, c = generate_problem_instance(m, n)

print("Matrix A:")
print(A)
print("Vector b:", b)
print("Vector c:", c)

#Solve with brute-force"
print("\nSolving with brute-force:")
bf_value, bf_solution = brute_force_knapsack(A, b, c)
print("Optimal value:", bf_value)
print("Optimal solution:", bf_solution)

Matrix A:
[[39 29 15 43  8 21 39 19 23 11 11 24 36 40 24]
 [ 3 22  2 24 44 30 38  2 21 33 12 22 44 25 49]
 [27 42 28 16 15 47 44  3 37  7 21  9 39 18  4]
 [25 14  9 26  2 20 28 47  7 44  8 47 35 14 17]
 [36 40  4  2  6 42  4 29 18 26 44 34 10 36 14]
 [31 48 15  8 14 23 40 21 16 45 18 47 24 26 25]
 [45 41 29 15 45  1 25  7  9 24  1 44  8 24 11]
 [17  8 35 35 33  5 42 39 41 28  7  9  8 12 34]
 [33 48 23 24 37 35 44 40 22 27 35  1 35 37 47]
 [14  3  1  5 26 14 39 27  9 15 15 26 42 13 32]]
Vector b: [61 13 32 24 52 38 45 22 41 80]
Vector c: [59 86 28 66 42 45 62 57  6 28 28 44 84 30 62]

Solving with brute-force:
Optimal value: 28
Optimal solution: [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0]


In [88]:
test_cases = [
    (np.array([[1]]), np.array([1]), np.array([10])),
    (np.array([[1, 2]]), np.array([3]), np.array([10, 15])),
    (np.array([[1, 2, 3]]), np.array([5]), np.array([10, 15, 40])),
    (np.array([[2, 3, 1, 4, 5], [3, 2, 2, 5, 4]]), np.array([10, 8]), np.array([20, 30, 40, 10, 50])),
    (np.array([[1, 1, 1, 1, 1, 1]]), np.array([3]), np.array([5, 4, 3, 2, 1, 10])),
    (np.array(np.random.randint(1, 5, size=(3, 12))), np.array([10, 15, 20]), np.array(np.random.randint(1, 100, size=12))),
    (np.array([np.ones(15)]), np.array([20]), np.array(np.random.randint(1, 50, size=15))),
    (np.array([[5, 6, 7, 8, 9, 10, 11, 12],[5, 6, 7, 8, 9, 10, 11, 12]]), np.array([3, 3]), np.array([10, 20, 30, 40, 50, 60, 70, 80])),
    (np.array([[10, 20, 30, 40, 50]]), np.array([5]), np.array([100, 200, 300, 400, 500])),
    (np.array([[1, 1, 1, 1, 1]]), np.array([5]), np.array([10, 20, 30, 40, 50]))
]

optimal_value = [10, 25, 55, 120, 19, -200, 99999, 0, 0, 150]
optimal_solutions = [[1], [1,1], [0,1,1], [0,1,1,0,1], [1,1,0,0,0,1], [-200], [1,1,1], [0,0,0,0,0,0,0,0], [0,0,0,0,0], [1,1,1,1,1]]

for i, (A, b, c) in enumerate(test_cases):
    print(f"Test {i+1}:")
    value, x = brute_force_knapsack(A, b, c)
    print("Optimal value:", value)
    print("And it should be:", optimal_value[i])
    print("Solution:", x)
    print("And it should be:", optimal_solutions[i])
    print("-" * 30)

Test 1:
Optimal value: 10
And it should be: 10
Solution: [1]
And it should be: [1]
------------------------------
Test 2:
Optimal value: 25
And it should be: 25
Solution: [1, 1]
And it should be: [1, 1]
------------------------------
Test 3:
Optimal value: 55
And it should be: 55
Solution: [0, 1, 1]
And it should be: [0, 1, 1]
------------------------------
Test 4:
Optimal value: 120
And it should be: 120
Solution: [0, 1, 1, 0, 1]
And it should be: [0, 1, 1, 0, 1]
------------------------------
Test 5:
Optimal value: 19
And it should be: 19
Solution: [1, 1, 0, 0, 0, 1]
And it should be: [1, 1, 0, 0, 0, 1]
------------------------------
Test 6:
Optimal value: 339
And it should be: -200
Solution: [1, 0, 0, 1, 0, 1, 0, 0, 1, 1, 1, 0]
And it should be: [-200]
------------------------------
Test 7:
Optimal value: 321
And it should be: 99999
Solution: [1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1]
And it should be: [1, 1, 1]
------------------------------
Test 8:
Optimal value: 0
And it shoul